In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras_spiking
import tensorflow as tf

In [2]:
# Load CIFAR 10
(
    (train_images, train_labels),
    (test_images, test_labels),
) = tf.keras.datasets.mnist.load_data()

In [3]:
# normalize images so values are between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

topologies = [
    [64, 64],
    [32],
    [32, 32],
    [64],
    [128],
]

def generate_non_spiking_model(topology):
    layers = [tf.keras.layers.Flatten(input_shape=(28, 28))]
    for hidden_layer_size in topology:
        layers.append(tf.keras.layers.Dense(hidden_layer_size, activation="relu"))
    layers.append(tf.keras.layers.Dense(10))
    model = tf.keras.Sequential(layers)
    return model

def generate_spiking_model(topology):
    layers = [tf.keras.layers.Reshape((-1, 28 * 28), input_shape=(None, 28, 28))]
    for hidden_layer_size in topology:
        layers.append(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(hidden_layer_size)))
        layers.append(keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True))
    layers.append(tf.keras.layers.GlobalAveragePooling1D(),)
    layers.append(tf.keras.layers.Dense(10))
    model = tf.keras.Sequential(layers)
    return model

def train(input_model, train_x, test_x):
    input_model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    
    num_epochs = 40
    best_test_acc = 0
    for num_epoch in range(num_epochs):
        input_model.fit(train_x, train_labels, epochs=1)
        _, test_acc = input_model.evaluate(test_x, test_labels, verbose=2)
        if test_acc > best_test_acc:
            best_epoch_model = tf.keras.models.clone_model(input_model)
            best_test_acc = test_acc
    return test_acc, best_epoch_model

In [5]:
n_steps = 10
train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1))
test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1))
models = dict()

In [6]:
for topology in topologies:
    print(str(topology))
    spiking_model = generate_spiking_model(topology)
    test_acc, best_model = train(spiking_model, train_sequences, test_sequences)
    models[str(topology)] = best_model
    print(test_acc)
    energy = keras_spiking.ModelEnergy(best_model, example_data=np.ones((1, 28, 28)))
    summ = energy.summary_string(
        columns=("name", "energy cpu", "energy loihi"),
        timesteps_per_inference=10,
        print_warnings=False,
        dt=0.01
    )
    loihi_energy = float(summ.split(':')[-1].strip())
    print(loihi_energy)

[64, 64]
1875/1875 [==============================] - 48s 25ms/step - loss: 3.1145 - accuracy: 0.4648
313/313 - 3s - loss: 0.6894 - accuracy: 0.7988
1875/1875 [==============================] - 46s 25ms/step - loss: 0.5808 - accuracy: 0.8220
313/313 - 3s - loss: 0.4736 - accuracy: 0.8491
1875/1875 [==============================] - 45s 24ms/step - loss: 0.4512 - accuracy: 0.8609
313/313 - 3s - loss: 0.3954 - accuracy: 0.8813
1875/1875 [==============================] - 46s 24ms/step - loss: 0.3830 - accuracy: 0.8816
313/313 - 3s - loss: 0.3398 - accuracy: 0.8940
1875/1875 [==============================] - 48s 26ms/step - loss: 0.3401 - accuracy: 0.8964
313/313 - 3s - loss: 0.2974 - accuracy: 0.9099
1875/1875 [==============================] - 45s 24ms/step - loss: 0.3065 - accuracy: 0.9067
313/313 - 3s - loss: 0.2765 - accuracy: 0.9154
1875/1875 [==============================] - 44s 23ms/step - loss: 0.2857 - accuracy: 0.9130
313/313 - 3s - loss: 0.2566 - accuracy: 0.9224
1875/1875 [

1875/1875 [==============================] - 31s 16ms/step - loss: 0.1794 - accuracy: 0.9446
313/313 - 2s - loss: 0.1870 - accuracy: 0.9430
1875/1875 [==============================] - 31s 16ms/step - loss: 0.1733 - accuracy: 0.9452
313/313 - 2s - loss: 0.1834 - accuracy: 0.9451
1875/1875 [==============================] - 31s 16ms/step - loss: 0.1716 - accuracy: 0.9478
313/313 - 2s - loss: 0.1754 - accuracy: 0.9464
1875/1875 [==============================] - 29s 16ms/step - loss: 0.1664 - accuracy: 0.9493
313/313 - 2s - loss: 0.1684 - accuracy: 0.9489
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1609 - accuracy: 0.9495
313/313 - 2s - loss: 0.1857 - accuracy: 0.9455
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1611 - accuracy: 0.9507
313/313 - 2s - loss: 0.1713 - accuracy: 0.9493
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1572 - accuracy: 0.9521
313/313 - 2s - loss: 0.1737 - accuracy: 0.9498
1875/1875 [=========

1875/1875 [==============================] - 44s 23ms/step - loss: 0.1746 - accuracy: 0.9472
313/313 - 3s - loss: 0.1978 - accuracy: 0.9387
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1726 - accuracy: 0.9469
313/313 - 3s - loss: 0.1929 - accuracy: 0.9399
1875/1875 [==============================] - 44s 23ms/step - loss: 0.1722 - accuracy: 0.9480
313/313 - 3s - loss: 0.1740 - accuracy: 0.9512
1875/1875 [==============================] - 45s 24ms/step - loss: 0.1678 - accuracy: 0.9493
313/313 - 3s - loss: 0.1831 - accuracy: 0.9457
1875/1875 [==============================] - 44s 23ms/step - loss: 0.1680 - accuracy: 0.9485
313/313 - 3s - loss: 0.1726 - accuracy: 0.9482
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1615 - accuracy: 0.9500
313/313 - 3s - loss: 0.1861 - accuracy: 0.9438
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1611 - accuracy: 0.9509
313/313 - 3s - loss: 0.1684 - accuracy: 0.9490
1875/1875 [=========

1153/1875 [=================>............] - ETA: 12s - loss: 0.2561 - accuracy: 0.9225

KeyboardInterrupt: 

In [ ]:
asdf = {
    "64,64": 94.83,
    "32": 94.65,
    "32, 32": 93.60,
    "64": 95.97,
    "128": 96.12
}

In [ ]:
topologies_2 = 